## COMA ASAS-SN SkyPatrol Comet Ingest Pipeline

The pyasassn client allows users to query the ASAS-SN input catalog and retrieve light curves from our database. These light curves are subject to live updates as we are running continuous photometry on our nightly images.



### ASAS-SN comets

Use the client.comets catalog to enumerate ASAS-SN comets

In [3]:
!pip3 install astroquery

  Using cached astroquery-0.4.7-py3-none-any.whl (5.3 MB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached keyring-25.3.0-py3-none-any.whl (38 kB)
  Using cached pyvo-1.5.2-py3-none-any.whl (910 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 10.2 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 4.4 MB/s eta 0:00:00


In [53]:
import os
import json
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from astropy.time import Time
#from astroquery.mpc import MPC
import pandas as pd
%matplotlib inline

from COMAJSONServer import COMAAPI

from pyasassn.client import SkyPatrolClient

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

client = SkyPatrolClient()
client.catalogs

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.




Table Name:  stellar_main
Num Columns: 47
Num Targets: 98932961

Table Name:  master_list
Num Columns: 4
Num Targets: 109300848

Table Name:  asassn_discoveries
Num Columns: 11
Num Targets: 5716

Table Name:  comets
Num Columns: 1
Num Targets: 14815

Table Name:  swift
Num Columns: 56
Num Targets: 254936

Table Name:  allwiseagn
Num Columns: 15
Num Targets: 1354900

Table Name:  mdwarf
Num Columns: 32
Num Targets: 8927

Table Name:  glade_tmassx
Num Columns: 20
Num Targets: 1125414

Table Name:  glade_gwgc
Num Columns: 20
Num Targets: 52399

Table Name:  m_giants
Num Columns: 12
Num Targets: 4879809

Table Name:  glade_pgc
Num Columns: 20
Num Targets: 52400

Table Name:  glade_hyperleda
Num Columns: 20
Num Targets: 2459906

Table Name:  glade_sdssqso
Num Columns: 20
Num Targets: 297152

Table Name:  milliquas
Num Columns: 21
Num Targets: 1979676

Table Name:  fermi
Num Columns: 67
Num Targets: 5788

Table Name:  aavsovsx
Num Columns: 28
Num Targets: 1437528

Table Name:  morx
Num Colu

In [5]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

#### Comets Catalog

The __comets__ catalog contains the comet targets that are of interest to COMA. 

In [6]:
client.catalogs.comets.head()

,col_names,dtypes
0,mpc_entry,string


In [7]:
client.random_sample(100, catalog="comets")

,mpc_entry
0,C/2020R7(ATLAS)MPEC2023-E49
1,C/2013V2(Borisov)MPC94279
2,C/2020U4(PANSTARRS)MPEC2022-S20
3,C/2019T4(ATLAS)MPEC2023-K53
4,C/2022E3(ZTF)MPEC2023-E49
...,...
95,C/2020O2(Amaral)MPEC2021-W31
96,C/2021T2(Fuls)MPEC2021-Y10
97,398P/BoattiniMPEC2020-W26
98,P/2021U3(Attard-Maury)MPEC2022-A21


#### The Master List

The __master_list__ contains __asas_sn_ids__ coordinates and catalog sources for all of our targets. All of our catalogs are cross-matched on the master list with a 2-arcsecond cone. 

In [8]:
client.catalogs.master_list

,col_names,dtypes
0,asas_sn_id,bigint
1,ra_deg,double
2,dec_deg,double
3,catalog_sources,array<string>


### Random Curves 

For whatever reason, if you are interested in random targets from a given catalog, we can give you those too.

In [9]:
client.random_sample(100, catalog="comets")

,mpc_entry
0,P/2016A7(PANSTARRS)MPC100580
1,P/2011R3(Novichonok-Gerke)MPC87476
2,263P/GibbsMPEC2023-O39
3,C/2018KJ3(Lemmon)MPEC2021-R75
4,C/2012A2(LINEAR)MPEC2013-N23
...,...
95,142P/Ge-WangMPC133426
96,P/2012T3(PANSTARRS)MPC80900
97,C/2018U1(Lemmon)MPC118094
98,P/2023V2(PANSTARRS)MPEC2023-X98


## Pick a comet from the list

-  417P is a periodic comet
-  C/2020K1 is a long period comet
#comet_regexp = 'C/2017K2.*'
-  P/2021L4 is a periodic comet

In [66]:
#comet_name = 'P/2016 A7'
#comet_name = '2P/EnckeMPEC2023'
#comet_name = 'C/2022 A2'
#comet_name = 'C/2022 P1'
#comet_name = 'C/2022 E3'
#comet_name = 'C/2023 E1'
#comet_name = 'C/2022 A2'
#comet_name = 'C/2021 X1'
#comet_name = 'C/2021 T4'
#comet_name = 'C/2020 V2'
#comet_regexp = 'C/2017 K2.*'

#comet_regexp = '417P.*'
#comet_regexp = 'C/2020 K1.*'
#comet_regexp = 'P/2021 L4.*'


#comet_name = 'C/2019 U5'
#comet_name = 'C/2021 X1'
#comet_name = 'C/2021 T4'
#comet_name = 'C/2019 T2'
comet_name = 'C/2017 K2'
#comet_name = '121P'
#comet_name = '12P'
#comet_name = '2P'


### ADQL Queries

We have included a custom ADQL parser. That will allow users to query targets using this familiar SQL-like language. 
Let's use ADQL to find all the observations of the comet we are interested in!


In [67]:
comet_regexp = comet_name.replace(' ','') + '%'
query = """
SELECT mpc_entry FROM comets WHERE mpc_entry LIKE '%s'
"""
query = query %(comet_regexp)
query

"\nSELECT mpc_entry FROM comets WHERE mpc_entry LIKE 'C/2017K2%'\n"

In [68]:
client.adql_query(query)

,mpc_entry
0,C/2017K2(PANSTARRS)MPC105544
1,C/2017K2(PANSTARRS)MPC106346
2,C/2017K2(PANSTARRS)MPC108595
3,C/2017K2(PANSTARRS)MPC109144
4,C/2017K2(PANSTARRS)MPC109591
...,...
74,C/2017K2(PANSTARRS)MPEC2023-V39
75,C/2017K2(PANSTARRS)MPEC2023-W26
76,C/2017K2(PANSTARRS)MPEC2023-XP6
77,C/2017K2(PANSTARRS)MPEC2023-Y97


In [69]:
lcs = client.adql_query(query, download=True, threads=8)

Pulled 64 of 79


In [70]:
lcs.data.head()

,mpc_entry,jd,flux,flux_err,mag,mag_err,limit,flux_3,flux_err_3,mag_3,...,flux_8,flux_err_8,mag_8,mag_err_8,limit_8,fwhm,image_id,camera,quality,phot_filter
0,C/2017K2(PANSTARRS)MPC105544,2.458029e+06,0.103202,0.044779,18.024953,99.999,18.024953,0.062255,0.065635,17.609808,...,-0.160349,0.172340,16.561678,99.999,16.561678,1.49,bq002227,bq,G,g
1,C/2017K2(PANSTARRS)MPC105544,2.458030e+06,0.017998,0.060507,17.698121,99.999,17.698121,0.190950,0.091684,17.246908,...,-0.079199,0.239187,16.205795,99.999,16.205795,1.59,bq002396,bq,G,g
2,C/2017K2(PANSTARRS)MPC105544,2.458018e+06,0.087709,0.076446,17.503969,99.999,17.503969,0.027438,0.111055,17.098507,...,0.480856,0.296662,16.031703,99.999,16.031703,1.69,ba439095,ba,B,V
3,C/2017K2(PANSTARRS)MPC105544,2.458019e+06,0.251671,0.075869,17.512198,99.999,17.512198,0.484740,0.112873,17.080880,...,1.100469,0.290327,16.055137,99.999,16.055137,1.71,ba439432,ba,G,V
4,C/2017K2(PANSTARRS)MPC105544,2.458023e+06,-0.060777,0.084409,17.396390,99.999,17.396390,-0.300948,0.129410,16.932438,...,0.000310,0.334965,15.899858,99.999,15.899858,1.79,ba440934,ba,G,V


In [71]:
lcs.data.describe()

,jd,flux,flux_err,mag,mag_err,limit,flux_3,flux_err_3,mag_3,mag_err_3,...,flux_7,flux_err_7,mag_7,mag_err_7,limit_7,flux_8,flux_err_8,mag_8,mag_err_8,limit_8
count,6.000000e+02,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,2.459166e+06,14.987102,0.152752,15.489868,35.363146,17.059543,24.092772,0.205793,15.053878,36.194559,...,54.221143,0.379230,14.300899,41.355869,16.001538,59.683603,0.419042,14.201406,43.186690,15.886830
std,7.272055e+02,26.037418,0.127256,2.315870,47.817621,0.897845,43.220906,0.166853,2.378408,48.066644,...,101.804199,0.286530,2.411251,49.264496,0.804680,115.892633,0.314070,2.401783,49.553860,0.795163
min,2.457903e+06,-92.433716,0.017982,11.350110,0.004945,15.205476,-210.444060,0.028582,10.814666,0.003926,...,-266.893377,0.061559,9.904664,0.002697,14.085535,-467.566697,0.070089,9.783524,0.002567,13.954477
25%,2.458595e+06,0.180624,0.055865,13.240989,0.012763,16.290539,0.217519,0.078600,12.782190,0.010728,...,0.283834,0.165093,12.277902,0.010259,15.349095,0.283193,0.186328,12.238108,0.010506,15.261417
50%,2.459316e+06,0.869281,0.105322,16.375725,0.084041,17.101476,1.206192,0.145322,16.027843,0.082276,...,1.983262,0.280099,15.305209,0.111314,16.037236,2.102942,0.304045,15.262695,0.115898,15.945296
75%,2.459784e+06,18.351043,0.221916,17.510882,99.999000,17.791777,28.004833,0.292189,17.150261,99.999000,...,44.552662,0.526525,16.428240,99.999000,16.610880,46.215067,0.573005,16.280848,99.999000,16.490178
max,2.460322e+06,104.708609,0.600988,19.015535,99.999000,19.015535,171.458757,0.809742,18.512423,99.999000,...,396.424815,1.685941,17.679413,99.999000,17.679413,443.217145,1.902241,17.530545,99.999000,17.538508


In [72]:
j = lcs.data['jd']
m = lcs.data['mag']
t_jd = Time(j, format='jd')
t_utc = t_jd.to_value('iso', 'date_hms')  
t_ymd = t_jd.to_value('datetime64', 'date')  
t_mjd = t_jd.to_value('mjd')  
lcs.data['utc'] = t_utc
lcs.data['date'] = t_ymd
lcs.data['mjd'] = t_mjd
date_form = DateFormatter("%y-%m")
print(lcs.data['mjd'])

0      58028.085234
1      58029.069169
2      58017.232107
3      58018.224295
4      58022.213972
           ...     
595    60309.408431
596    60308.275650
597    60311.219493
598    60294.158177
599    60321.848724
Name: mjd, Length: 600, dtype: float64


In [73]:
filtered = lcs.data[lcs.data['mag_err'] < 0.5] 
print(filtered.shape)

(388, 45)


In [74]:
mpc_comet = comet_name
mpc_start_ts = filtered.describe()['date']['min']
td = pd.Timedelta(1, "min")
mpc_end_ts = mpc_start_ts + td
print(mpc_start_ts)
print(mpc_end_ts)
mpc_start = mpc_start_ts.strftime('%Y-%m-%dT%H:%M:%S')
print(mpc_start)
mpc_end = mpc_end_ts.strftime('%Y-%m-%dT%H:%M:%S')
print(mpc_end)

2018-04-08 00:00:00
2018-04-08 00:01:00
2018-04-08T00:00:00
2018-04-08T00:01:00


In [75]:
print(filtered.describe()['date']['min'])
print(filtered.describe()['date']['max'])

2018-04-08 00:00:00
2024-01-12 00:00:00


In [76]:
filtered.sort_values(by='mjd', inplace = True)
print(filtered['mjd'])

43     58216.338052
48     58226.324987
254    58243.274823
56     58282.271101
57     58316.278068
           ...     
594    60305.278114
596    60308.275650
595    60309.408431
597    60311.219493
599    60321.848724
Name: mjd, Length: 388, dtype: float64


/tmp/ipykernel_31/372987866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.sort_values(by='mjd', inplace = True)


In [77]:
assdf_g = filtered[filtered['phot_filter']=='g']
assdf_g['object'] = assdf_g['mpc_entry']
assdf_g['telescope'] = 'ASAS-SN'
assdf_g['filter'] = 'G_ASASSN'
assdf_g['aperture'] = 16
assdf_g['zp_mag'] = assdf_g['limit']
assdf_g['zp_mag_err'] = float('nan')	
assdf_g['phase_angle'] = float('nan')	
assdf_g['heliocentric_au'] = float('nan')	
assdf_g['geocentric_au'] = float('nan')	
assdf_g['true_anomaly'] = float('nan')
assdf_g = assdf_g.drop(['mpc_entry', 'limit', 'jd', 'flux', 'flux_err', 'fwhm', 'image_id','camera','quality','phot_filter'], axis=1)
assdf_g = assdf_g.drop(['flux_3','flux_err_3','mag_3','mag_err_3','limit_3'], axis=1)
assdf_g = assdf_g.drop(['flux_4','flux_err_4','mag_4','mag_err_4','limit_4'], axis=1)
assdf_g = assdf_g.drop(['flux_5','flux_err_5','mag_5','mag_err_5','limit_5'], axis=1)
assdf_g = assdf_g.drop(['flux_6','flux_err_6','mag_6','mag_err_6','limit_6'], axis=1)
assdf_g = assdf_g.drop(['flux_7','flux_err_7','mag_7','mag_err_7','limit_7'], axis=1)
assdf_g = assdf_g.drop(['flux_8','flux_err_8','mag_8','mag_err_8','limit_8'], axis=1)

/tmp/ipykernel_31/3268152417.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assdf_g['object'] = assdf_g['mpc_entry']
/tmp/ipykernel_31/3268152417.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assdf_g['telescope'] = 'ASAS-SN'
/tmp/ipykernel_31/3268152417.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

# Save ASAS-SN data to csv file

In [78]:
filtered.head()

,mpc_entry,jd,flux,flux_err,mag,mag_err,limit,flux_3,flux_err_3,mag_3,...,mag_err_8,limit_8,fwhm,image_id,camera,quality,phot_filter,utc,date,mjd
43,C/2017K2(PANSTARRS)MPC109591,2.458217e+06,0.285351,0.049449,17.761617,0.188359,17.917257,0.362265,0.069068,17.502498,...,99.999,16.581478,1.57,bq062395,bq,G,g,2018-04-08 08:06:47.727,2018-04-08,58216.338052
48,C/2017K2(PANSTARRS)MPC109591,2.458227e+06,0.338407,0.060967,17.576468,0.195826,17.689897,0.383255,0.085583,17.321666,...,99.999,16.324887,1.67,bt064932,bt,G,g,2018-04-18 07:47:58.877,2018-04-18,58226.324987
254,C/2017K2(PANSTARRS)MPEC2018-HA5,2.458244e+06,0.313763,0.053256,17.658560,0.184493,17.836718,0.362395,0.073834,17.481998,...,99.999,16.490486,1.55,bt068946,bt,G,g,2018-05-05 06:35:44.716,2018-05-05,58243.274823
56,C/2017K2(PANSTARRS)MPC110495,2.458283e+06,0.188613,0.032548,18.211135,0.187569,18.371340,0.250684,0.043619,17.902251,...,99.999,17.180226,1.58,bs078662,bs,G,g,2018-06-13 06:30:23.109,2018-06-13,58282.271101
57,C/2017K2(PANSTARRS)MPC110495,2.458317e+06,0.215769,0.032176,18.065092,0.162090,18.383807,0.250694,0.040884,17.902207,...,99.999,17.254442,1.44,bs085518,bs,G,g,2018-07-17 06:40:25.084,2018-07-17,58316.278068


In [79]:
file_name = comet_name.replace('/','-').replace(' ','_')
assdf_g.to_csv('ASASSN_'+file_name+'.csv', index=False,
    columns = ['object', 'telescope', 'filter', 'aperture', 'utc', 'mjd', 'mag', 'mag_err', 'zp_mag', 'zp_mag_err', 'phase_angle', 'heliocentric_au', 'true_anomaly', 'geocentric_au'])